In [137]:
pip install dash==2.8.1

Note: you may need to restart the kernel to use updated packages.


In [138]:
!pip install jupyter-dash



In [139]:
data = pd.read_csv("avocado.csv")
data

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,7,2018-02-04,1.63,17074.83,2046.96,1529.20,0.00,13498.67,13066.82,431.85,0.0,organic,2018,WestTexNewMexico
18245,8,2018-01-28,1.71,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,organic,2018,WestTexNewMexico
18246,9,2018-01-21,1.87,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,organic,2018,WestTexNewMexico
18247,10,2018-01-14,1.93,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,organic,2018,WestTexNewMexico


In [140]:
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output

# Load full dataset (no filter here!)
data = pd.read_csv("avocado.csv")
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")

# Get all unique regions and types
regions = sorted(data["region"].unique())

# Create the Dash app
app = JupyterDash(__name__, title="Avocado Dashboard")

# App layout
app.layout = html.Div([
    html.H1("Avocado Analytics", style={"fontSize": "48px", "color": "red"}),

    html.P("Analyze the behavior of avocado prices and the number "
           "of avocados sold in the US between 2015 and 2018"),

    # Region Dropdown
    html.Div([
        html.Label("Select Region:"),
        dcc.Dropdown(
            id="region-dropdown",
            options=[{"label": r, "value": r} for r in regions],
            value="Albany",  # default region
            clearable=False
        ),
    ], style={"width": "300px", "margin-bottom": "20px"}),

    # Line chart
    dcc.Graph(id="price-chart")
])

# Callback to update graph based on region
@app.callback(
    Output("price-chart", "figure"),
    Input("region-dropdown", "value")
)
def update_chart(region):
    filtered_data = data.query("type == 'conventional' and region == @region")
    fig = {
        "data": [{
            "x": filtered_data["Date"],
            "y": filtered_data["AveragePrice"],
            "type": "line",
            "name": region
        }],
        "layout": {
            "title": f"Average Avocado Price Over Time in {region}",
            "xaxis": {"title": "Date"},
            "yaxis": {"title": "Price ($)"},
        }
    }
    return fig

# Run the app inside the notebook
app.run_server(mode='inline', debug=True)


In [141]:
regions = data["region"].unique()
avocado_types = data["type"].unique()

In [142]:
app.layout = html.Div(
    children=[
        html.H1("Avocado Analytics"),
        html.P("Analyze the behavior of avocado prices and the number "
               "of avocados sold in the US between 2015 and 2018"),

        # Filter Menu
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div("Region", className="menu-title"),
                        dcc.Dropdown(
                            id="region-filter",
                            options=[
                                {"label": region, "value": region}
                                for region in regions
                            ],
                            value="Albany",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div("Type", className="menu-title"),
                        dcc.Dropdown(
                            id="type-filter",
                            options=[
                                {
                                    "label": avocado_type.title(),
                                    "value": avocado_type,
                                }
                                for avocado_type in avocado_types
                            ],
                            value="organic",
                            clearable=False,
                            searchable=False,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div("Date Range", className="menu-title"),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed=data["Date"].min().date(),
                            max_date_allowed=data["Date"].max().date(),
                            start_date=data["Date"].min().date(),
                            end_date=data["Date"].max().date(),
                        ),
                    ]
                ),
            ],
            className="menu",  # You can style this with external CSS later
        ),

        # Placeholder for Graphs
        html.Div(
            children=dcc.Graph(id="price-chart"),
            className="chart"
        )
    ]
)


In [143]:
from dash.dependencies import Input, Output

@app.callback(
    Output("price-chart", "figure"),
    Output("volume-chart", "figure"),
    Input("region-filter", "value"),
    Input("type-filter", "value"),
    Input("date-range", "start_date"),
    Input("date-range", "end_date"),
)
def update_charts(region, avocado_type, start_date, end_date):
    # Filter the dataset using user inputs
    filtered_data = data.query(
        """
        region == @region and type == @avocado_type
        and Date >= @start_date and Date <= @end_date
        """
    )

    # Optional safety check if no data matches
    if filtered_data.empty:
        return {
            "data": [],
            "layout": {
                "title": {"text": "No data available for selected filters."}
            },
        }, {
            "data": [],
            "layout": {
                "title": {"text": "No data available for selected filters."}
            },
        }

    # Price Chart Figure
    price_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["AveragePrice"],
                "type": "lines",
                "hovertemplate": "$%{y:.2f}<extra></extra>",
            },
        ],
        "layout": {
            "title": {
                "text": "Average Price of Avocados",
                "x": 0.05,
                "xanchor": "left",
            },
            "xaxis": {"fixedrange": True, "title": "Date"},
            "yaxis": {"tickprefix": "$", "fixedrange": True, "title": "Price ($)"},
            "colorway": ["#17B897"],
        },
    }

    # Volume Chart Figure
    volume_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["Total Volume"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {
                "text": "Total Avocados Sold",
                "x": 0.05,
                "xanchor": "left",
            },
            "xaxis": {"fixedrange": True, "title": "Date"},
            "yaxis": {"fixedrange": True, "title": "Volume"},
            "colorway": ["#E12D39"],
        },
    }

    return price_chart_figure, volume_chart_figure


In [144]:
html.Div([
    html.H1("Avocado Analytics"),
    html.P("Analyze the behavior of avocado prices..."),

    html.Div([
        html.Div([
            html.Div("Region", className="menu-title"),
            dcc.Dropdown(..., className="dropdown"),
        ]),
        html.Div([
            html.Div("Type", className="menu-title"),
            dcc.Dropdown(..., className="dropdown"),
        ]),
        html.Div([
            html.Div("Date Range", className="menu-title"),
            dcc.DatePickerRange(...),
        ]),
    ], className="menu"),

    html.Div([
        dcc.Graph(id="price-chart")
    ], className="chart")
])


Div([H1('Avocado Analytics'), P('Analyze the behavior of avocado prices...'), Div(children=[Div([Div(children='Region', className='menu-title'), Dropdown(options=Ellipsis, className='dropdown')]), Div([Div(children='Type', className='menu-title'), Dropdown(options=Ellipsis, className='dropdown')]), Div([Div(children='Date Range', className='menu-title'), DatePickerRange(start_date=Ellipsis)])], className='menu'), Div(children=[Graph(id='price-chart')], className='chart')])

In [145]:
dcc.Dropdown(
    id="region-filter",
    options=[{"label": region, "value": region} for region in regions],
    value="Albany",  # default value
    clearable=False,
    className="dropdown",
)


Dropdown(options=[{'label': 'Albany', 'value': 'Albany'}, {'label': 'Atlanta', 'value': 'Atlanta'}, {'label': 'BaltimoreWashington', 'value': 'BaltimoreWashington'}, {'label': 'Boise', 'value': 'Boise'}, {'label': 'Boston', 'value': 'Boston'}, {'label': 'BuffaloRochester', 'value': 'BuffaloRochester'}, {'label': 'California', 'value': 'California'}, {'label': 'Charlotte', 'value': 'Charlotte'}, {'label': 'Chicago', 'value': 'Chicago'}, {'label': 'CincinnatiDayton', 'value': 'CincinnatiDayton'}, {'label': 'Columbus', 'value': 'Columbus'}, {'label': 'DallasFtWorth', 'value': 'DallasFtWorth'}, {'label': 'Denver', 'value': 'Denver'}, {'label': 'Detroit', 'value': 'Detroit'}, {'label': 'GrandRapids', 'value': 'GrandRapids'}, {'label': 'GreatLakes', 'value': 'GreatLakes'}, {'label': 'HarrisburgScranton', 'value': 'HarrisburgScranton'}, {'label': 'HartfordSpringfield', 'value': 'HartfordSpringfield'}, {'label': 'Houston', 'value': 'Houston'}, {'label': 'Indianapolis', 'value': 'Indianapolis'}

In [146]:
# Load full data first
full_data = pd.read_csv("avocado.csv")
full_data["Date"] = pd.to_datetime(full_data["Date"])

# Extract all regions and types from full data
regions = full_data["region"].unique()
avocado_types = full_data["type"].unique()

# Filter for default display (you can change "Albany" to any region later)
data = (
    full_data
    .query("type == 'conventional' and region == 'Albany'")
    .sort_values(by="Date")
)


In [147]:
regions = full_data["region"].unique()
regions

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

In [162]:
# Run the app inside the notebook
app.run_server(mode='inline', debug=True)
